# Ingest data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
# data = pd.read_csv("./dataset/diabetes.csv")
data = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")

In [ ]:
data.columns

# Analysis

## 1) Data Exploration

In [ ]:
data.shape

Lets take a look at the data to determine if we have any null values and the datatypes associated to each column.
It is important to deal with null values and categorical columns as machine learning algorithms dont do well with sparse dataset and understand only numeric values.
In case we find any categorical values we will need to convert them to numeric and also perform one hot encoding in  order to remove bias.
In our dataset we have all numeric columns and there are no null values present in the dataset.

In [ ]:
data.info()

### Check how many other missing(zero) values

In [ ]:
print("total number of rows : {0}".format(len(data)))
print("number of rows missing Glucose: {0}".format(len(data.loc[data['Glucose'] == 0])))
print("number of rows missing BloodPressure: {0}".format(len(data.loc[data['BloodPressure'] == 0])))
print("number of rows missing SkinThickness: {0}".format(len(data.loc[data['SkinThickness'] == 0])))
print("number of rows missing Insulin: {0}".format(len(data.loc[data['Insulin'] == 0])))
print("number of rows missing BMI: {0}".format(len(data.loc[data['BMI'] == 0])))
print("number of rows missing DiabetesPedigreeFunction: {0}".format(len(data.loc[data['DiabetesPedigreeFunction'] == 0])))
print("number of rows missing Age: {0}".format(len(data.loc[data['Age'] == 0])))

In [ ]:
features_data = data[data.columns[:-1]]
total = (features_data==0).sum().sort_values(ascending=False)
percent = (((features_data==0).sum()/(features_data==0).count())*100).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
f, ax = plt.subplots(figsize=(15, 6))
plt.xticks(rotation='90')
sns.barplot(x=missing_data.index, y=missing_data['Percent'])
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)
missing_data.head()

In [ ]:
data.describe()

describe() gives out a lot of information.
* Number of rows and columns in the dataset
* A number of summary statistics about the dataset such as
     * Minimum value
     * Mean value
     * Maximum value
     * Standard deviation value
     * Percentiles


We can now take a look at some sample records in our dataset.

In [ ]:
data.head()

## 2) Visualizing Data

In [ ]:
data.hist(figsize=(10,10))

We can detect skewness in our dataset using skew function in scipy.stats

In [ ]:
from scipy.stats import skew
data.iloc[:, :-1].apply(lambda x: skew(x.dropna().astype(float)))

As verified using our histograms of individual feature distribution and the scipy stats function insulin, DiabetesPedigreeFunction, Age and Pregnancy are positively skewed

Now let us also look at the correlation of the individual features. We can use the corr() function. We can use pair grid to visualize the distribution of features in the dataset corresponding to each other and the outcomes.

In [ ]:
plt.figure(figsize=(15,15))
corr = data.corr()
corr.index = data.columns
sns.heatmap(corr, annot = True, cbar=True, vmin=-1, vmax=1, square = True)
plt.title("Correlation Heatmap", fontsize=16)
plt.show()

 We didnt find any strong correlation between a person being diabetic and any independent variable as a result we cannot remove any independent variable from our analysis.

# Baseline models

### Using Cross Validation
Many a times, the data is imbalanced, i.e there may be a high number of class1 instances but less number of other class instances. Thus we should train and test our algorithm on each and every instance of the dataset. Then we can take an average of all the noted accuracies over the dataset.
- The K-Fold Cross Validation works by first dividing the dataset into k-subsets.
- Let's say we divide the dataset into (k=5) parts. We reserve 1 part for testing and train the algorithm over the 4 parts. 
- We continue the process by changing the testing part in each iteration and training the algorithm over the other parts. The accuracies and errors are then averaged to get a average accuracy of the algorithm. This is called K-Fold Cross Validation.
- An algorithm may underfit over a dataset for some training data and sometimes also overfit the data for other training set. Thus with cross-validation, we can achieve a generalised model.

In [ ]:
from sklearn.model_selection import StratifiedKFold

features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI', 'DiabetesPedigreeFunction', 'Age']
outcomes = ['Outcome']

X = data[features]
y = data[outcomes]

skf = StratifiedKFold(n_splits=10, random_state=10)

In [ ]:
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

model_names = ["Linear SVM","Radial SVM", "Logistic Regression", "Decision Tree"]
models = [svm.SVC(kernel="linear"), svm.SVC(kernel="rbf"), LogisticRegression(), DecisionTreeClassifier(criterion="entropy")]

mean_accuracy = []
mean_f1 = []
global_accuracy = []
global_f1 = []

def evaluate_models(X=X, y=y):
   '''
       This method performs 10 fold cross validation on list of models and returns a dataframe with mean f1 and mean accuracy scores for each model
   '''
   for name, model in zip(model_names, models):
        accuracy=[] 
        f1 = []
        
        for train_index, test_index in skf.split(X, y):
            
            X_train = X.loc[train_index] 
            y_train = y.loc[train_index]
            X_test = X.loc[test_index]
            y_test = y.loc[test_index]
            
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            accuracy.append(metrics.accuracy_score(y_test, y_pred))
            f1.append(metrics.f1_score(y_test, y_pred))
            
        global_accuracy.append(accuracy)
        global_f1.append(f1)
        mean_accuracy.append(np.mean(np.array(accuracy)))
        mean_f1.append(np.mean(np.array(f1)))
        
   model_perf_df = pd.DataFrame(np.array([mean_accuracy, mean_f1]).T,index=model_names)   
   model_perf_df.columns = ['Mean Accuracy', 'Mean F1']
   return model_perf_df



In [ ]:
def generate_box_plot():
    '''Generate box plots to visualize the Mean Accuracy and Mean F1 scores calculated across multiple models'''
    box=pd.DataFrame(data=global_accuracy,index=[model_names])
    plt.figure(figsize=(20, 20))
    sns.boxplot(data=box.T).set_title('Mean Accuracy')
    
    box=pd.DataFrame( data=global_f1,index=[model_names])
    plt.figure(figsize=(20, 20))
    sns.boxplot(data=box.T).set_title('Mean F1 score')

In [ ]:
## define utility methods
def convert_to_dataframe(ndar, cols):
    '''Given a set of records in nupy array and a list of column names return a Pandas dataframe'''
    pdf = pd.DataFrame.from_records(ndar)
    pdf.columns = cols
    return pdf

In [ ]:
evaluate_models().sort_values(ascending=False, by = 'Mean F1')

In [ ]:
generate_box_plot()

# Preprocessing Data

## 1) Impute missing values

In [ ]:
features_no_pregnancy_no_skinthickness =  features.copy()
features_no_pregnancy_no_skinthickness.remove('Pregnancies')
features_no_pregnancy_no_skinthickness.remove('SkinThickness')
features_no_pregnancy = features_no_pregnancy_no_skinthickness.copy()
features_no_pregnancy.append('SkinThickness')

In [ ]:
from sklearn.impute import SimpleImputer as  Imputer

fill_values = Imputer(missing_values=0, strategy="mean")
X[features_no_pregnancy_no_skinthickness] = fill_values.fit_transform(X[features_no_pregnancy_no_skinthickness])

fill_values = Imputer(missing_values=0, strategy="median")
X[features_no_pregnancy] = fill_values.fit_transform(X[features_no_pregnancy])

## 2) Standardisation
There can be a lot of deviation in the given dataset. An example in the dataset can be the BMI where it has 248 unique values. This high variance has to be standardised. Standardization is a useful technique to transform attributes with a Gaussian distribution and differing means and standard deviations to a standard Gaussian distribution with a mean of 0 and a standard deviation of 1.

In [ ]:
# Feature scaling with StandardScaler
from sklearn.preprocessing import StandardScaler
scale_features_std = StandardScaler()
X = scale_features_std.fit_transform(X)

## 3) Stratification:
When we split the dataset into train and test datasets, the split is completely random. Thus the instances of each class label or outcome in the train or test datasets is random. Thus we may have many instances of class 1 in training data and less instances of class 2 in the training data. So during classification, we may have accurate predictions for class1 but not for class2. Thus we stratify the data, so that we have proportionate data for all the classes in both the training and testing data.

In [ ]:
X = convert_to_dataframe(X, features)

In [ ]:
X.hist(figsize=(10,10))

In [ ]:
X.skew()

# Model Training


### Model performance after preprocessing data.


In [ ]:
mean_accuracy = []
mean_f1 = []
global_accuracy = []
global_f1 = []
evaluate_models().sort_values(ascending=False, by = 'Mean F1')

## Identifying most important features with the Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
model= RandomForestClassifier(n_estimators=250,random_state=10)
model.fit(X,y)
pd.Series(model.feature_importances_,index=X.columns).sort_values(ascending=False)

The important features are: Glucose, BMI, Age, DiabetesPedigreeFunction. Taking only the important features and rerunning the model training

# Baseline model summary with top features

In [ ]:
most_important_features = ["Glucose", "BMI", "Age", "DiabetesPedigreeFunction" ] 
X_most_important = X[most_important_features]
y_most_important = y[outcomes]

In [ ]:
mean_accuracy = []
mean_f1 = []
global_accuracy = []
global_f1 = []
evaluate_models(X=X_most_important, y=y_most_important)

In [ ]:
generate_box_plot()

Taking into consideration the top 4 features made the  performance improvement and mean accuracy improved by certain points.

# Advanced Modelling techniques to improve model performance

### Ensembling
Ensemble methods are techniques that create multiple models and then combine them to produce improved results. Ensemble methods usually produces more accurate solutions than a single model would. The models used to create such ensemble models are called ‘base models’.

We will do ensembling with the Voting Ensemble. Voting is one of the simplest ways of combining the predictions from multiple machine learning algorithms. It works by first creating two or more standalone models from your training dataset. A Voting Classifier can then be used to wrap your models and average the predictions of the sub-models when asked to make predictions for new data.

We will be using weighted Voting Classifier. We will assign to the classifiers according to their accuracies. So the classifier with single accuracy will be assigned the highest weight and so on.

In our case, we will use the Top 3 classifiers i.e Linear SVM, Radial SVM and Logistic Regression classifiers.

#### Logistic regression, Linear SVM, Radial SVM

In [ ]:
from sklearn.ensemble import VotingClassifier #for Voting Classifier

In [ ]:
radial_svc=svm.SVC(kernel='rbf', probability=True)
linear_svc=svm.SVC(kernel='linear', probability=True)
lr=LogisticRegression()

In [ ]:
ensemble_lin_rad_lr=VotingClassifier(estimators=[('Linear_svm', linear_svc), ('Radial_SVM', radial_svc),('Logistic Regression', lr)], voting='soft', weights=[0.2, 0.1, 0.1])

In [ ]:
model_names.append("Logistic regression, Linear, radial SVM Ensemble model")
models.append(ensemble_lin_rad_lr)

### XGBoost

In [ ]:
import sys
# !{sys.executable} -m pip install xgboost
!{sys.executable} -m conda install -y -c  anaconda py-xgboost

In [ ]:
## Hyper Parameter Optimization
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
}

In [ ]:
## Hyperparameter optimization using RandomizedSearchCV and getting the best estimator using cross validation
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

In [ ]:
from sklearn.metrics import make_scorer
classifier = xgb.XGBClassifier()
random_search = RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='f1',n_jobs=-1,cv=10,verbose=3, random_state=10 )
random_search.fit(X, y)
#get best estimator
random_search.best_estimator_

Using the best estimator to define a classifier

In [ ]:
classifier=xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, gamma=0.2,
              learning_rate=0.05, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
model_names.append("Xgboost")
models.append(classifier) 

In [ ]:
mean_accuracy = []
mean_f1 = []
global_accuracy = []
global_f1 = []
evaluate_models(X=X_most_important, y=y_most_important).sort_values(ascending=False, by = 'Mean F1')

In [ ]:
generate_box_plot()